Наброски кода для
0. сведения алломорфов морфем в одно (хотя бы в таблице)
1. подсчёта сочетаний морфем
2. вытаскивания конвербов
3. вытаскивания цепочек частей речи

Все манипуляции можно тестировать на корпусах, которые в почте.

# 0
Действительно надо вытащить всё, положить в словарину и радоваться

Формат:

`корпус{
    название:документ{    # done
        мета:мета,    # done
        текст:[
            предложение{
                слой:[
                    морфемы
                    ],
                перевод:''
            }
        ]
    }
}`

In [1]:
corpora = ['Kamchatka', 'Sebjan']

In [5]:
import os, re, pickle
from pprint import pprint


def morphs_2_words(line):
    '''берёт на вход массив расчленённой строки, возвращает массив слов'''
    words = []
    word = []
    for morph in line:
        if morph.strip()[0] in '=-':
            word.append(morph)
        else:
            if len(word) > 0:
                words.append(word)
            word = [morph]
    words.append(word)
    return words


def handle_startline(line, res, current_layer):
    line = line.split()
    layer = line[0].strip('\\')
    if not (len(line) == 1): # чтобы не считать пустые строки
        if layer in parted_layers:
            line_content = morphs_2_words(line[1:]) # делим на слова, состоящие из морфем
        else:
            line_content = [' '.join(line[1:])] # просто целые строки (комментарии и тп)
            current_layer = layer
        if layer in res and res[layer][0] != '':
            res[layer] += line_content
        else:
            res[layer] = line_content
    return res, current_layer


def lines_2_dict(part): # FIX EMPTY LINES
    '''
    i: кусок текста (предложение) в несколько строк, в каждой строке несколько слоёв, и с другими данными предложения
    o: джейсонина вида {'слой': [сл, о, ва], 'слой': содержимое}
    доп. ограничения: длина всех строк-массивов равна
    '''
    res = {}
    lines = [line for line in part.split('\n') if len(line) > 1]
    res['index'] = [lines[0].split('_')[-1]]
    parted_layers = ['tx', 'mb', 'ge', 'ps']
    current_layer = '' # для переносов
    for line in lines[1:]:
        if line.startswith('\\'):
            res, current_layer = handle_startline(line, res, current_layer)
        else:
            if current_layer:
                res[current_layer][0] += ' ' + line
    return res


def check_len(p_sent, fil):
    parted_layers = ['mb', 'ge', 'ps']
    selected_layers = [key for key in p_sent if key in parted_layers and len(p_sent[key])>1]
    # 1. check that the number of words is the same
    lengths = set([len(p_sent[key]) for key in selected_layers])
    if len(lengths) > 1:
        print('Error in {}, here:'.format(fil))
        print(lengths)
        for l in selected_layers:
            print(len(p_sent[l]))
            pprint(p_sent[l])
        return lengths
    # 2. check that morphemes are aligned
    if 'mb' in selected_layers and 'ge' in selected_layers:
        for i in range(len(p_sent['ge'])):
            if len(p_sent['ge'][i]) != len(p_sent['mb'][i]):
                print('што-то слиплось в {}'.format(fil))
                pprint(p_sent)
    return lengths


def make_readable(corp):
    "переводит текст корпусов в удобомашиночитаемую джейсонину"
    folder = 'Corpus_Text_{}_postagged'.format(corp)
    corpus_dict = {}
    for fil in os.listdir(folder):
        if not fil.endswith('.txt'):
            continue
        with open(os.path.join(folder, fil), 'r') as f:
            text = f.read()
        file_content, text_content = {}, []
        sents = text.split('\id')
        file_content['meta'] = sents[1] # metainfo at the beginning of the file; not parsed
        for sent in sents[2:]:
            sent_content = lines_2_dict(sent)
            check_len(sent_content, fil)
            text_content.append(sent_content)
        file_content['text'] = text_content
        corpus_dict[fil] = file_content
    with open('{}.pickle'.format(corp), 'wb') as f:
        pickle.dump(corpus_dict, f)
    return corpus_dict

In [2]:
# for corp in corpora:
#     make_readable(corp)

In [3]:
def detect(what, where, corpora=corpora, show=True):
    for corpus in corpora:
        hits = []
        with open('{}.pickle'.format(corpus), 'rb') as f:
            content = pickle.load(f)
        for doc in content:
            for i in range(len(content[doc]['text'])): # looping through sentences
                if not where in content[doc]['text'][i]:
                    continue
                for word in content[doc]['text'][i][where]:
                    if what in word:
                        print('Found in {}, {} at {}'.format(corpus, doc, i))
                        if show:
                            pprint(content[doc]['text'][i])
                        hits.append(doc)
        if len(hits) == 0:
            print('noth found. cool! or not?')
        else:
            inp = input('correct all these ({} hits) & press Enter and I\'ll reload.\n'.format(len(hits)))
            if 'replace' in inp:
                target = inp.split()[1]
                for doc in list(set(hits)):
                    with open(os.path.join('Corpus_Text_{}_postagged'.format(corpus), doc), 'r') as f:
                        text = f.read()
                    text = re.sub('{} '.format(what), '{} '.format(target), text)
                    with open(os.path.join('Corpus_Text_{}_postagged'.format(corpus), doc), 'w') as f:
                        f.write(text)
            make_readable(corpus)
            print('done!')

In [9]:
detect('=2sg\ft', 'ge')

noth found. cool! or not?
noth found. cool! or not?


In [94]:
# with open('{}.pickle'.format('Sebjan'), 'rb') as f:
#     content = pickle.load(f)
# pprint(content['Alekseeva_RD_lost_tapes_znatoki_NA.txt']['text'][55]['ge'])
fil = 'Kejmetinova_AA_headmistress_Yakutsk_310310_LZ.txt'
with open(os.path.join('Corpus_Text_Sebjan_postagged', fil), 'r') as f:
    text = f.read()
sents = text.split('\id')
for i in range(len(sents)):
    sent_content = lines_2_dict(sents[i])
    check_len(sent_content, fil)
# part = sents[27]
# res = {}
# lines = [line for line in part.split('\n') if len(line) > 1]
# res['index'] = [lines[0].split('_')[-1]]
# parted_layers = ['tx', 'mb', 'ge', 'ps']
# current_layer = '' # для переносов
# for line in lines[1:]:
#     if line.startswith('\\'):
#         line = line.split()
#         layer = line[0].strip('\\')
#         if not (len(line) == 1): # чтобы не считать пустые строки
#             print(line)
#             if layer in parted_layers:
#                 line_content = morphs_2_words(line[1:]) # делим на слова, состоящие из морфем
#             else:
#                 line_content = [' '.join(line[1:])] # просто целые строки (комментарии и тп)
#                 current_layer = layer
#             if layer in res and res[layer][0] != '':
#                 res[layer] += line_content
#             else:
#                 res[layer] = line_content
#     else:
#         if current_layer:
#             res[current_layer][0] += ' ' + line
# pprint(res)

# 1
это уже начато там
морфемы лежат в отдельном словаре, кажется; нужно приложить это знание на таблицу?
или создать уже наконец промежуточное представление всего

# 2
сочетания морфем

сначала надо вытащить представления (кажется это уже сделано)

пройтись по всему, растащить каждое слово на пары морфем, первая (ROOT, morph), последняя (morph, END)

`from collections import Counter
# вытаскиваю все морфемы из первого слота, считаю, с чем они сочитаются, записываю в словарь
morph_pairs_dict = {x: Counter([pair for pair in morph_pairs_list if pair[0]==x]) for x in set(map(lambda pair: pair[0], morph_pairs_list))}
`
